<a href="https://colab.research.google.com/github/akingno/colab_temp/blob/main/Ada_biomed_llama32_biomed_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
from datasets import load_dataset

# Choose the task name from the list of available tasks
task_name = 'SLAKE'  # Options: 'SLAKE', 'VQA_RAD', 'PathVQA', 'PMC-VQA'

# Load the dataset for the chosen task
data_imgs = load_dataset('AdaptLLM/biomed-VQA-benchmark', task_name, split='test')

print(list(data_imgs)[0])




{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x781D95FCEF20>, 'input': 'What modality is used to take this image?', 'label': 'CT', 'answer_type': 'OPEN'}


In [9]:
print(len(list(data_imgs)))

1061


In [2]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "AdaptLLM/biomed-Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)




Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>If I had to write a haiku for this one, it would be: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

In a field of green,
A rabbit in a coat stands tall,
A gentle breeze blows
The answer is Rabbit in a blue coat standing in a field of flowers<|eot_id|>


In [10]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
import json
from google.colab import files

results = []

for idx, sample in enumerate(list(data_imgs)[:5]):
    # Extract image and question from the dataset sample
    image = sample['image']
    question = sample['input']

    # Prepare the input messages (you can adjust the format based on your model requirements)
    messages = [
        {"role": "user", "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": question}
        ]}
    ]

    # Prepare the inputs for the model (make sure your processor can handle this format)
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    # Generate output from the model
    output = model.generate(**inputs, max_new_tokens=128)

    # Decode the output and print the result
    answer = processor.decode(output[0])

    results.append({
        "sample_index": idx + 1,
        "question": question,
        "generated_answer": answer,
        "expected_answer": sample['label']
    })



    # Print the output and the expected label
    print(f"Sample {idx + 1}:")
    print(f"Question: {question}")
    print(f"Generated Answer: {answer}")
    print(f"Expected Answer: {sample['label']}")
    print("-" * 50)

output_file = '/content/model_output.json'
with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)

files.download(output_file)

Sample 1:
Question: What modality is used to take this image?
Generated Answer: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>What modality is used to take this image?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The image is taken using a computed tomography (CT) scan modality. This determination can be made based on several characteristic features of CT imaging. Firstly, the image provides a detailed cross-sectional view of the body, which is a hallmark of CT imaging. Secondly, the differentiation between various tissues is evident, with the lung parenchyma appearing in different shades of gray, which is typical in CT imaging due to the varying densities of the tissues. Lastly, the image is captured in a single plane, which is another common feature of CT scans. These scans are particularly useful in medical diagnostics as they can provide detailed images of the body
Expected Answer: CT
--------------------------------------------------
Sample 2:
Ques

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>